**月報**https://mops.twse.com.tw/mops/web/t21sc04_ifrs

In [1]:
from excellib import set_excel_color_daily
from stocklib import get_stock_price, get_OTC_price, get_otc, get_investor, get_ratio
from stocklib import cal_bar, cal_KD, check_10days, check_10days_otc, get_trust_today
from stocklib import convert_num
from stock_plot import plot_K_chart


from finance import get_yearly, get_df_monthly, get_df_monthly_check, get_df_quarterly
from finance import get_this_quarter, get_quarter_income, get_quarter_finance, set_growth_eps
from finance import get_df_quarterly_all, get_df_yearly_all

In [2]:
import requests
import shutil
import time
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import yfinance as yf
import json
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

ua = UserAgent()
root = '../'
path_trust = root + 'trust/'
path_check = root + 'daily_check/'
path_investor = root + 'investor/'
path_flow = root + 'flow/'

# 取得今天日期

In [3]:
d_today = date.today() #- timedelta(days=1)

today_tw = d_today - relativedelta(years=1911)
this_year = d_today.year
this_quarter = get_this_quarter()
n = 1
if d_today.weekday() == 0:
    n = 3
d_yesterday = d_today - timedelta(days=n)
s_yesterday = (d_today - timedelta(days=n)).strftime("%Y%m%d")
s_today_md = (d_today).strftime("%m/%d")
s_yesterday_md = (d_today - timedelta(days=n)).strftime("%m/%d")
s_today = d_today.strftime("%Y%m%d")

s_today_tw = today_tw.strftime("%Y/%m/%d").lstrip('0')
s_thismonth_tw = today_tw.strftime("%Y/%m").lstrip('0')
print('今天:', d_today, '昨天:', s_yesterday)
# 台灣證券交易所

今天: 2022-09-28 昨天: 20220927


In [4]:
df_yesterday = get_stock_price(d_yesterday)

df_today = get_stock_price(d_today)
df_today.head(2)

取得今日上市股票資訊: 20220927
取得今日上市股票資訊: 20220928


,name,amount,open,high,low,close
id,,,,,,
0050,元大台灣50,20345,107.60,107.65,105.25,105.25
0051,元大中型100,53,51.95,51.95,50.40,51.75


In [5]:
df_investor = get_investor(d_today)

lst_investor = list(df_investor[ (df_investor['investor']>0) & (df_investor['trust']>0)].index)
print('外資, 投信同步買超筆數：', len(lst_investor))
print(list(lst_investor))
df_investor.head(2)

外資, 投信同步買超筆數： 11491
外資, 投信同步買超筆數： 52
['5880', '3481', '2834', '2845', '2880', '2367', '2886', '2892', '1216', '3045', '2412', '4904', '3231', '2449', '6770', '3703', '6176', '2912', '2455', '2615', '2633', '3042', '6531', '2206', '2454', '3661', '2379', '1304', '2838', '4977', '4915', '3454', '3023', '9941', '4958', '1309', '6239', '1802', '3665', '8016', '1773', '8454', '2395', '2850', '8110', '1229', '8081', '2520', '1733', '6719', '2637', '2607']


,investor,trust,dealer_T,dealer_F
id,,,,
00632R,-1681,0,-9084,87955
00637L,2752,0,453,14399


In [6]:
lst_investor_trust = list(df_investor[ (df_investor['investor']>100) | (df_investor['trust']>100)].index)
print('外資 or 投信買超筆數：', len(lst_investor_trust))

外資 or 投信買超筆數： 193


# 取得交易價量前30名

In [7]:
df_amt = df_today.copy()
df_amt['value_amt'] = df_amt['amount'] * df_amt['close'] # 成交量 * 收盤價(億)
df_amt['value_amt'] //= 100000
df_amt.sort_values('value_amt', ascending=False, inplace=True)
df_amt = df_amt[df_amt['close'] > df_amt['open']]
lst_amt = list(df_amt.head(30).index)
print('today:', lst_amt)

path_amt30 = './amt30.txt'
with open(path_amt30, 'r') as f:
    lst_amt30_yesterday = f.read().split('\n')
print('yesterday:', lst_amt30_yesterday)

with open(path_amt30, 'w') as f:
    for stock_id in lst_amt:
        f.write(stock_id + "\n")
lst_amt30_new = [i for i in lst_amt if i not in lst_amt30_yesterday]
print('\nnew:', lst_amt30_new)
df_amt = df_amt.head(30)
df_amt_new = df_amt.reset_index()
df_amt_new[df_amt_new['id'].isin(lst_amt30_new)]

today: ['3046', '00632R', '1216', '5880', '2912', '3045', '2382', '2395', '4904', '2364', '2356', '2347', '00673R', '4720', '2753', '6505', '00664R', '00671R', '08220P', '08213P', '08219P', '082206', '082183', '082184', '01004T', '082130', '08202P', '082034', '08204P', '08212P']
yesterday: ['2603', '1795', '3443', '2609', '3037', '6533', '2388', '8046', '1605', '1760', '3406', '2308', '8478', '1101', '3189', '3008', '00637L', '3708', '4919', '6285', '2383', '6443', '2345', '1529', '2497', '1536', '3046', '8996', '6669', '2206', '']

new: ['00632R', '1216', '5880', '2912', '3045', '2382', '2395', '4904', '2364', '2356', '2347', '00673R', '4720', '2753', '6505', '00664R', '00671R', '08220P', '08213P', '08219P', '082206', '082183', '082184', '01004T', '082130', '08202P', '082034', '08204P', '08212P']


,id,name,amount,open,high,low,close,value_amt
1,00632R,元大台灣50反1,293230,6.18,6.34,6.18,6.34,18.0
2,1216,統一,10984,66.30,67.00,66.00,66.60,7.0
3,5880,合庫金,23380,26.20,26.45,26.00,26.45,6.0
4,2912,統一超,2234,281.00,286.00,280.00,283.50,6.0
5,3045,台灣大,6203,97.10,97.70,96.00,97.40,6.0
6,2382,廣達,7276,77.00,77.70,76.30,77.70,5.0
7,2395,研華,1846,300.00,304.00,298.50,301.00,5.0
8,4904,遠傳,6811,71.60,72.50,70.80,72.30,4.0
9,2364,倫飛,4702,72.30,78.80,72.30,78.80,3.0
10,2356,英業達,14761,22.75,23.00,22.60,22.85,3.0


# 昨天外資、投信同步買超，檢查隔天是否有上漲

In [8]:
import yfinance as yf
def cal_mean_20(stock_id):
    data_yf = yf.Ticker(stock_id+'.TW')
    df = data_yf.history(period="1mo")
    df.sort_index(ascending=False, inplace=True)
    
    price_20 = round(np.mean(df['Close'][:20]),2)
    return price_20

In [9]:
# path_track_yesterday = path_investor + 'investors_2days_{}.xlsx'.format(s_yesterday)
# # shutil.copy2(file_track, file_track.replace('.xlsx', '_backup.xlsx')) 

# df_track_yes = pd.read_excel(path_track_yesterday, index_col=None, engine='openpyxl')
# df_track_yes['id'] = df_track_yes['id'].astype(str)
# lst_track_yes = list(set(df_track_yes['id']))
# print('昨天外資投信同步買超筆數：', len(lst_track_yes), s_yesterday)
# df_track_yes.set_index('id', inplace=True)
# df_track_close_yes = df_track_yes[['close']]
# df_track_close_yes.columns = ['close_yesterday']

# # 昨天外資、投信同步買進，只保留隔天有上漲的股票
# df_track_today = df_today[df_today.index.isin(lst_track_yes)].copy()
# df_track_today['increase'] = 0
# df_track_today['mean20'] = 0
# df_track_today = df_track_today.join(df_track_close_yes)
# df_track_today['increase'] = df_track_today['close'] - df_track_today['close_yesterday']
# df_track_today = df_track_today[df_track_today['increase']>=0]
# df_track_today['mean20'] = df_track_today.index.to_series().apply(cal_mean_20)
# # <月線2%內
# df_track_today = df_track_today[df_track_today['close'] < df_track_today['mean20'] * 1.02]
# lst_increase = list(df_track_today.index)
# print('隔天上漲且 <月線2%內筆數：', len(lst_increase), lst_increase)

# # join (今日股價、今日法人買賣超)
# df_track_today_inv = df_track_today.join(df_investor)
# df_track_today_inv.insert(1, 'date', s_today)
# df_track_today_inv

# # # save file
# df_track_yes = df_track_yes[df_track_yes.index.isin(lst_increase)]
# df_track_updated = pd.concat([df_track_yes.iloc[:,:-1],df_track_today_inv])
# df_track_updated.reset_index(inplace=True)
# df_track_updated.sort_values(['id', 'date'], ascending=[True, False], inplace=True)
# df_track_updated.to_excel(path_track_yesterday, index=False)
# df_track_updated

昨天外資投信同步買超筆數： 119 20220927
隔天上漲且 <月線2%內筆數： 3 ['2704', '3045', '4904']


,id,name,date,amount,open,high,low,close,increase,mean20,investor,trust,dealer_T,close_yesterday,dealer_F
3,2704,國賓,20220928,494,30.65,31.20,30.00,30.70,0.05,31.77,-214,1,0,30.65,0.0
0,2704,國賓,20220927,129,30.15,30.65,30.15,30.65,0.00,0.00,30,2,-1,NaN,NaN
4,3045,台灣大,20220928,6203,97.10,97.70,96.00,97.40,0.40,97.65,2908,574,-124,97.00,2.0
1,3045,台灣大,20220927,3807,96.70,97.20,96.20,97.00,0.00,0.00,195,1066,-31,NaN,NaN
5,4904,遠傳,20220928,6811,71.60,72.50,70.80,72.30,0.90,72.91,1744,473,3,71.40,-6.0
2,4904,遠傳,20220927,3153,71.00,71.60,70.40,71.40,0.00,0.00,577,563,9,NaN,NaN


# 今天外資、投信同步買超

In [10]:
path_track_today = path_investor + 'investors_2days_{}.xlsx'.format(s_today)

df_inv_price = df_today[df_today.index.isin(lst_investor)].copy()
df_inv_price['increase'] = 0
df_inv_price['mean20'] = 0

df_inv_today = df_inv_price.join(df_investor)
df_inv_today

print('今天外資投信同步買超筆數：', len(lst_investor), lst_investor)

# save file
df_inv_today.insert(1, 'date', s_today)
df_inv_today.reset_index(inplace=True)
df_inv_today.to_excel(path_track_today, index=False)
df_inv_today.head(2)

今天外資投信同步買超筆數： 52 ['5880', '3481', '2834', '2845', '2880', '2367', '2886', '2892', '1216', '3045', '2412', '4904', '3231', '2449', '6770', '3703', '6176', '2912', '2455', '2615', '2633', '3042', '6531', '2206', '2454', '3661', '2379', '1304', '2838', '4977', '4915', '3454', '3023', '9941', '4958', '1309', '6239', '1802', '3665', '8016', '1773', '8454', '2395', '2850', '8110', '1229', '8081', '2520', '1733', '6719', '2637', '2607']


,id,name,date,amount,open,high,low,close,increase,mean20,investor,trust,dealer_T,dealer_F
0,1216,統一,20220928,10984,66.3,67.0,66.0,66.6,0,0,3314,800,-160,80
1,1229,聯華,20220928,1693,52.9,52.9,51.1,51.1,0,0,14,9,0,8


## 外資、投信同步買超 (檢查交易量突破2倍、收盤價突破10天盤整)

In [11]:
## 今日符合篩選條件，新增至追蹤
# 漲幅 > 1.5%，K棒 > 1%
df_add = df_today.loc[lst_investor,:]
df_add.sort_index(inplace=True)

df_add['rise%'] = round((df_add['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
df_add['K%'] = round( (df_add['close'] - df_add['open'])/df_add['open']*100, 2 )    # K棒%
df_add = df_add[ (df_add['rise%']>1.5) & (df_add['K%']>1) ]
df_add_min = df_add.copy()
df_add_min_yes = df_add.copy()

df_add = check_10days(df_add)
lst_add_ori = list(df_add.index)
print('今日新增:',len(lst_add_ori),'筆\n', lst_add_ori)

lst_add = df_add.index
df_add = df_add.loc[lst_add,:]
df_add

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%
id,,,,,,,,


## 量價前30名 (檢查交易量突破2倍、收盤價突破10天盤整)

In [12]:
print('量價前30名:', lst_amt)
df_amt = check_10days(df_amt)
lst_amt = list(df_amt.index)
print('量價前30名，10日盤整:',len(lst_amt),'筆', lst_amt)
df_amt

量價前30名: ['3046', '00632R', '1216', '5880', '2912', '3045', '2382', '2395', '4904', '2364', '2356', '2347', '00673R', '4720', '2753', '6505', '00664R', '00671R', '08220P', '08213P', '08219P', '082206', '082183', '082184', '01004T', '082130', '08202P', '082034', '08204P', '08212P']
- 08220P.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 08213P.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 08219P.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 082206.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 082183.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
- 08202P.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
量價前30名，10日盤整: 0 筆 []


,name,amount,open,high,low,close,value_amt,check
id,,,,,,,,


## 外資 or 投信 買超

In [13]:
## 外資 or 投信 買超
df_inv_t = df_today.loc[lst_investor_trust,:]
df_inv_t.sort_index(inplace=True)

df_inv_t['rise%'] = round((df_inv_t['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
df_inv_t['K%'] = round( (df_inv_t['close'] - df_inv_t['open'])/df_inv_t['open']*100, 2 )    # K棒%
df_inv_t = df_inv_t[ (df_inv_t['rise%']>1) & (df_inv_t['K%']>1) ]
df_inv_min = df_inv_t.copy()
df_inv_min_yes = df_inv_t.copy()

df_inv_t = check_10days(df_inv_t)
lst_inv_t = list(df_inv_t.index)
print('今日新增:',len(lst_inv_t),'筆\n', lst_inv_t)

df_inv_t

- 06685P.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


## 外資、投信同步買超 (檢查創新低、昨日最低點)

In [14]:
# 今日創新低
df_add_min = check_10days(df_add_min, check='min')
lst_add_min = list(df_add_min.index)
print('今日新增:',len(lst_add_min),'筆\n', lst_add_min)
df_add_min

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%
id,,,,,,,,


In [15]:
# 昨日最低點
df_add_min_yes = check_10days(df_add_min_yes, check='yes')
lst_add_min_yes = list(df_add_min_yes.index)
print('今日新增:',len(lst_add_min_yes),'筆\n', lst_add_min_yes)
df_add_min_yes

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%
id,,,,,,,,


## 外資 or 投信 買超(檢查創新低、昨日最低點)

In [16]:
df_inv_min = check_10days(df_inv_min, check='min')
lst_inv_min = list(df_inv_min.index)
print('今日新增:',len(lst_inv_min),'筆\n', lst_inv_min)

df_inv_min

- 06685P.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


In [17]:
df_inv_min_yes = df_inv_t.copy()
df_inv_min_yes = check_10days(df_inv_min_yes, check='yes')
lst_inv_min_yes = list(df_inv_min_yes.index)
print('今日新增:',len(lst_inv_min_yes),'筆\n', lst_inv_min_yes)

df_inv_min_yes

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


# 下載Gmail股票交易excel

In [ ]:
from google_apis import create_service

CLIENT_FILE = 'client_secret.json'
API_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://mail.google.com/']

service = create_service(CLIENT_FILE, API_NAME, API_VERSION, SCOPES)

def search_emails(query_string, label_ids=None):
    try:
        message_list_response = service.users().messages().list(
                userId='me',
                labelIds=label_ids,
                q=query_string
        ).execute()
        message_items = message_list_response.get('messages')
        next_page_token = message_list_response.get('nextPageToken')
        return message_items
    except Exception as e:
        print(e)
        
def get_file_data(message_id, attachment_id, file_name, save_locatioin):
    response = service.users().messages().attachments().get(
        userId='me',
        messageId=message_id,
        id=attachment_id
    ).execute()
    
    file_data = base64.urlsafe_b64decode(response.get('data').encode('utf-8'))
    return file_data

def get_message_detail(message_id, msg_format='metadata', metadata_headers=None):
    message_detail = service.users().messages().get(
        userId='me',
        id=message_id,
        format=msg_format,
        metadataHeaders=metadata_headers
    ).execute()
    return message_detail

In [ ]:
import os
import base64
from tqdm import tqdm

def download_excel():
    query_string = 'has:attachment'
    save_folder = os.path.join(os.path.dirname(os.getcwd()), 'fugle')
    email_messages = search_emails(query_string)

    for email_message in tqdm(email_messages):
        message_detail = get_message_detail(email_message['id'], msg_format='full', metadata_headers=['parts'])
        if '富果帳戶 交易明細' in message_detail['snippet'] and not 'Forwarded' in message_detail['snippet']:
            message_detail_payload = message_detail.get('payload')
            if 'parts' in message_detail_payload:
                for msg_payload in message_detail_payload['parts']:
                    file_name = msg_payload['filename']
                    body = msg_payload['body']
                    if 'attachmentId' in body:
                        folder = file_name[4:10]
                        if file_name == '交易明細20220928.xlsx':
                            return
                        attachment_id = body['attachmentId']
                        attachment_content = get_file_data(email_message['id'], attachment_id, file_name, save_folder)
                        with open(os.path.join(save_folder, folder, file_name), 'wb') as f:
                            f.write(attachment_content)
                            print(f"File {file_name} is saved at {save_folder} {folder}")
download_excel()

# 上市上櫃 各類股 資金流向
**找出資金流向幅度增長最高的類股**

In [30]:
# 上市、上櫃資金流向表
url_flow = 'https://fubon-ebrokerdj.fbs.com.tw/z/zb/zba/zba.djhtm'
tables = pd.read_html(url_flow)
df = tables[-1]
df.head(3)

,0,1,2,3,4,5,6,7
0,上市資金流向表 最後更新時間: 09/28,上市資金流向表 最後更新時間: 09/28,上市資金流向表 最後更新時間: 09/28,上市資金流向表 最後更新時間: 09/28,上市資金流向表 最後更新時間: 09/28,上市資金流向表 最後更新時間: 09/28,上市資金流向表 最後更新時間: 09/28,上市資金流向表 最後更新時間: 09/28
1,類股名稱,流向率,類股名稱,流向率,類股名稱,流向率,類股名稱,流向率
2,水泥,0.85%,食品,0.51%,塑膠,0.92%,紡織纖維,0.56%


In [31]:
def get_stock_type_format(df):
    df_all = pd.DataFrame()
    for i in range(1, len(df.columns)//2 + 1):
        i_end = i * 2
        df_tmp = df.iloc[:, i_end-2:i_end]
        df_all = pd.concat([df_all, df_tmp])
    df_all.columns = ['type_name', 'flow_rate']
    df_all.dropna(inplace=True)
    df_all.reset_index(drop=True, inplace=True)
    df_all['flow_rate'] = df_all['flow_rate'].apply(lambda x: x.replace('%', ''))
    df_all['flow_rate'] = df_all['flow_rate'].astype(float)
    df_all = df_all.sort_values('flow_rate', ascending=False)
    
    df_all_t = df_all.T
    df_all_t.columns = df_all_t.iloc[0,:]
    df_all_t = df_all_t.iloc[1:,:]
    df_all_t.insert(0, 'date', value=s_today)
#     df_all_t.to_excel('stock_type.xlsx', index=False)
    return df_all_t

def get_flow_diff(df_new, df_old):
    df_new = df_new.iloc[:, 1:]
    df_new.reset_index(inplace=True, drop=True)
    
    df_yes = df_old.iloc[[-1], :]
    df_yes = df_yes.iloc[:, 1:]
    df_yes.reset_index(inplace=True, drop=True)
    
    df_diff = df_new.subtract(df_yes, fill_value=0)
    df_diff.insert(0, 'date', 'Difference')
    return df_diff

def cal_type_flow(path_file, df_new):
    print(path_file)
    df = pd.read_excel(path_flow + path_file, index_col=None, engine='openpyxl')
    df_ori = df.iloc[:-1, :]
    df_diff = get_flow_diff(df_new, df_ori)
    df_type = pd.concat([df_ori, df_new])
    df_type = pd.concat([df_type, df_diff])
    df_type.to_excel(path_flow + path_file, index=False)
    return df_type.reset_index(drop=True)

In [32]:
df_type = df.iloc[2:10, :]
df_type.columns = df.iloc[1,:]

df_type_new = get_stock_type_format(df_type)

cal_type_flow('stock_type.xlsx', df_type_new).iloc[-3:,:]

stock_type.xlsx


,date,電子,半導體,航運業,電子零組件,化學生技醫療,金融保險,化工,電腦及週邊設備,光電,...,紡織纖維,電子通路,食品,水泥,觀光,橡膠,油電燃氣,造紙,資訊服務,玻璃陶瓷
70,20220927,52.42,27.27,11.71,9.3,9.55,4.77,1.44,4.66,4.16,...,0.55,0.37,0.34,0.93,1.25,0.29,0.07,0.05,0.06,0.05
71,20220928,54.67,26.9,11.48,8.95,5.76,5.37,1.18,5.4,4.99,...,0.56,0.52,0.51,0.85,0.67,0.31,0.1,0.06,0.07,0.05
72,Difference,2.25,-0.37,-0.23,-0.35,-3.79,0.6,-0.26,0.74,0.83,...,0.01,0.15,0.17,-0.08,-0.58,0.02,0.03,0.01,0.01,0.0


In [33]:
df_otc = df.iloc[12:17, :]
df_otc.columns = df.iloc[11,:]

df_otc_new = get_stock_type_format(df_otc)

cal_type_flow('stock_type_otc.xlsx', df_otc_new).iloc[-3:, :]

stock_type_otc.xlsx


,date,電子,半導體,電子零組件,生技醫療,光電,通信網路,其他電子,電腦及週邊設備,電機機械,化工,文化創意,其他,觀光,資訊服務,鋼鐵,電子通路,航運業,建材營造,紡織纖維
70,20220927,56.64,33.19,5.56,31.39,6.6,3.74,3.49,3.29,1.17,2.01,0.92,1.59,1.73,0.22,1.69,0.54,0.12,0.1,0.05
71,20220928,63.54,31.08,7.06,25.67,10.2,4.44,4.29,5.65,2.15,1.48,0.89,1.7,1.43,0.23,1.26,0.6,0.24,0.14,0.08
72,Difference,6.9,-2.11,1.5,-5.72,3.6,0.7,0.8,2.36,0.98,-0.53,-0.03,0.11,-0.3,0.01,-0.43,0.06,0.12,0.04,0.03


# 畫日K線圖、週K線圖，儲存每天股票資料

In [22]:
file_name = root + 'stock_20210825_1203.json'
with open(file_name) as f:
    dic_stock = json.load(f)

latest_date_file = datetime.strptime(sorted(dic_stock)[-1], '%Y-%m-%d').date()
print('目前記錄日期至:',latest_date_file, '新增前:', len(dic_stock), '筆')

目前記錄日期至: 2022-09-27 新增前: 244 筆


In [23]:
import datetime
import time
import requests
from io import StringIO
import pandas as pd
import numpy as np

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    print(str(date).split(' ')[0].replace('-',''))
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret


data_stock = {}
date_today = d_today
n_days = (d_today - latest_date_file).days
fail_count = 0
allow_continuous_fail_count = 5
print('須補天數:', n_days)
while len(data_stock) < n_days:
    print('parsing', date_today)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        if date_today.weekday()  in [5,6]:
            print('Weekend!')
            date_today -= datetime.timedelta(days=1)
            continue
        data_stock[date_today] = crawl_price(date_today)
        if not str(date_today) in dic_stock.keys():
            dic_stock[str(date_today)] = data_stock[date_today].to_dict()
            print("add to dict:", str(date_today))
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
        break
    
    # 減一天
    date_today -= datetime.timedelta(days=1)
    time.sleep(10)

print('新增後:', len(dic_stock))
with open(file_name, 'w') as f:
    json.dump(dic_stock, f)

須補天數: 1
parsing 2022-09-28
20220928
add to dict: 2022-09-28
success!
新增後: 245


In [24]:
# def plot_daily_weekly_K(lst_stock_id, dic_stock):
#     df_close = pd.DataFrame({k:d['收盤價'] for k,d in dic_stock.items()}).transpose()
#     df_close.index = pd.to_datetime(df_close.index)

#     df_open = pd.DataFrame({k:d['開盤價'] for k,d in dic_stock.items()}).transpose()
#     df_open.index = pd.to_datetime(df_open.index)

#     df_high = pd.DataFrame({k:d['最高價'] for k,d in dic_stock.items()}).transpose()
#     df_high.index = pd.to_datetime(df_high.index)

#     df_low = pd.DataFrame({k:d['最低價'] for k,d in dic_stock.items()}).transpose()
#     df_low.index = pd.to_datetime(df_low.index)

#     df_volume = pd.DataFrame({k:d['成交股數'] for k,d in dic_stock.items()}).transpose()
#     df_volume.index = pd.to_datetime(df_volume.index)
    
#     # =========================== 畫K線圖 =================================================
#     for stock_id in tqdm(lst_stock_id):
#         thisyear = '2022'
#         print(stock_id)
#         if not stock_id in df_close.keys():
#             print(stock_id, '查無資料')
#             continue
#     #     stock_id = '2603'
#         dic_a_stock = {
#             'close':df_close[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'open':df_open[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'high':df_high[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'low':df_low[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'volume': df_volume[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#         }

#         # Daily DataFrame
#         df_stock = pd.DataFrame(dic_a_stock)
#         df_stock.insert(0,'stock_id', stock_id)
#         df_stock.sort_index(inplace=True)

#         fig_daily = plot_K_chart(df_stock, 'daily')

#         # Weekly DataFrame
#         period_type = '1W'
#         df_week = df_stock.resample(period_type).last()
#         df_week['close'] = df_stock['close'].resample(period_type).last()
#         df_week['open'] = df_stock['open'].resample(period_type).first()
#         df_week['high'] = df_stock['high'].resample(period_type).max()
#         df_week['low'] = df_stock['low'].resample(period_type).min()
#         df_week['volume'] = df_stock['volume'].resample(period_type).sum()

#         fig_weekly = plot_K_chart(df_week, 'weekly')

In [27]:
# df_close = pd.DataFrame({k:d['收盤價'] for k,d in dic_stock.items()}).transpose()
# df_close.index = pd.to_datetime(df_close.index)

# df_open = pd.DataFrame({k:d['開盤價'] for k,d in dic_stock.items()}).transpose()
# df_open.index = pd.to_datetime(df_open.index)

# df_high = pd.DataFrame({k:d['最高價'] for k,d in dic_stock.items()}).transpose()
# df_high.index = pd.to_datetime(df_high.index)

# df_low = pd.DataFrame({k:d['最低價'] for k,d in dic_stock.items()}).transpose()
# df_low.index = pd.to_datetime(df_low.index)

# df_volume = pd.DataFrame({k:d['成交股數'] for k,d in dic_stock.items()}).transpose()
# df_volume.index = pd.to_datetime(df_volume.index)

# # =========================== 畫K線圖 =================================================
# stock_id = '2603'
# thisyear = '2022'
# print(stock_id)
# if not stock_id in df_close.keys():
#     print(stock_id, '查無資料')
# #     stock_id = '2603'
# dic_a_stock = {
#     'close':df_close[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'open':df_open[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'high':df_high[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'low':df_low[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'volume': df_volume[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
# }

# # Daily DataFrame
# df_stock = pd.DataFrame(dic_a_stock)
# df_stock.insert(0,'stock_id', stock_id)
# df_stock.sort_index(inplace=True)
# df_stock

In [26]:
# print('今日新增 && 在量價前30名', list(set(lst_add_ori)& set(lst_amt)) )
# print('昨天外資投信同步買超筆數 & 今天上漲：', len(lst_increase), '筆')
# plot_daily_weekly_K(lst_increase, dic_stock)

In [28]:
# print('今日觀察：', len(lst_add_ori), '筆')
# plot_daily_weekly_K(lst_add_ori, dic_stock)

In [29]:
# print('量價前30名：', len(lst_amt),'筆\n', lst_amt)
# lst_amt_without_add = [i for i in lst_amt if i not in lst_add_ori]
# plot_daily_weekly_K(lst_amt_without_add, dic_stock)

# 預測EPS

In [18]:
# def get_EPS(lst_stock_id, filename):
#     df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
#     df_quarterly_all = get_df_quarterly_all()
#     df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])

#     # 取得公司發行股數
#     path_company_stock_num = root + 'company_list.xlsx'
#     df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
#     df_company_stock_num.set_index('stock_id', inplace=True)
    
#     lst_eps = []
#     for i in range(len(lst_stock_id)):
#         stock_id = lst_stock_id[i]
#         t_wait = np.random.randint(5,15)
#         print(stock_id, ', wait ', t_wait)
#         df = get_df_quarterly(stock_id)

#         if df is None:
#             print('df is None')
#             continue
#         if df.loc[df.index[0], '毛利率(%)'] < 20:
#             print(stock_id, '：毛利率', df.loc[df.index[0], '毛利率(%)'], '< 20%，跳過')
#             continue
#         elif df.loc[df.index[0], '營益率(%)'] < 8:
#             print(stock_id, '：營益率' , df.loc[df.index[0], '營益率(%)'] , ' < 8%，跳過')
#             continue

#         lst_eps.append(stock_id)
#         # 計算歷年同季的營收成長率、EPS
#         df = set_growth_eps(df, df_today, stock_id)

#         # 將年度EPS寫到歷年excel裡
#         lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
#         min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
#         lst_year = [y for y in lst_year if int(y) > min_year]
#         lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

#         df.reset_index(inplace=True)
#         df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

#         stock_name = ""
#         if stock_id in df_today.index:
#             stock_name = df_today.loc[stock_id, 'name']
#             df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    
#             df_yearly_all = df_yearly_all.append(df_yearly, ignore_index=True)

#         df.insert(0, 'name', value=stock_name)
#         df.insert(0, 'stock_id', value=stock_id)

#         df_quarterly_all = df_quarterly_all.append(df, ignore_index=True)
#         time.sleep(t_wait)

#         # ===============================
#         # 預測EPS ( check_EPS )
#         # ===============================
#         df_new_quarter = get_quarter_finance(stock_id, df, this_year, this_quarter, df_company_stock_num, df_today)
#         df_concat = pd.concat([df, df_new_quarter])
#         df_concat.sort_values('quarter', ascending=False, inplace=True)
#         df_estimate = df_estimate.append(df_concat)

#     #預測EPS ( check_EPS )
#     df_estimate.to_excel(path_check+'estimate_{}_{}.xlsx'.format(filename, s_today), index=False)
#     return lst_eps